In [1]:
import requests
import json
import pandas as pd
import time
import convex as cx
import tmqaalpha as tm

In [2]:
f_test = open("/data/users/romain.claret/tm/wikidata-simplequestions/annotated_wd_data_test.txt",'r')
out_test = f_test.readlines()

In [3]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/wikidata-simplequestions/benchmark_pickles/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [4]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    response = (json.loads(query.json()
            .get("questions")[0]
            .get("question")
            .get("answers")
            .replace('\n', ''))
     .get("results").get("bindings"))
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")

In [5]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                else: return False
            else: return False
        else:
            if '@id' in (json.dumps(response.json()['member']['result'])):
                ps_result = (json.dumps(response.json()["member"]['result']['@id']))
            else: return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")

In [6]:
def ask_convex(question):
    #initial_question = "Which actor voiced the Unicorn in The Last Unicorn?"
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    if not cx_result:
        return cx_result
    if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
        return False
    return cx_result

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")

In [7]:
def ask_mine(question):
    return tm.answer_initial_question(question)[1]
#ask_mine("Which actor voiced the Unicorn in The Last Unicorn?")

'Q202725'

In [ ]:
# Evaluate qanswer
#arguments[0] Subject
#arguments[1] Predicate
#arguments[2] Object
#arguments[3] question

#HEADERS = ['question', 'source', 'qanswer', 'platypus', 'convex', 'mine']
#df = pd.DataFrame(columns=HEADERS)

start_time = time.time()
out_test_len = len(out_test)
print(out_test_len)
for i, line in enumerate(out_test):
    arguments = line.split("\t")
    print("\r\t>>> Processing {}/{}: {}".format(i,out_test_len,arguments[3][:-1]), end='                                  ')
    if i <= 408:
        continue
    result_qanswer = ask_qanswer(arguments[3][:-1])
    result_platypus = ask_platypus(arguments[3][:-1])
    result_convex = ask_convex(arguments[3][:-1])
    result_mine = ask_mine(arguments[3][:-1])[1]
    print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == arguments[2] else False
    print("\n",i, "-> Platypus", result_platypus) if result_platypus == arguments[2] else False
    print("\n",i, "-> Convex", result_convex) if result_convex == arguments[2] else False
    print("\n",i, "-> Mine", result_mine) if result_mine == arguments[2] else False
    df = df.append({"question":arguments[3][:-1], 'source':arguments[2], 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex, 'mine':result_mine}, ignore_index=True)

end_time = time.time()
print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
print(df)

9961


	>>> Processing 410/9961: what is the country from which james c. dobbin nationality is derived                                  


 410 -> qAnswer Q30

 410 -> Convex Q30
	>>> Processing 412/9961: Who produced the ladykillers?                                                
 412 -> qAnswer Q317567
	>>> Processing 413/9961: Is ricardo delgado male of female                                  
 413 -> Convex Q6581097
	>>> Processing 416/9961: what is constantinos mina gender                                                          
 416 -> qAnswer Q6581097

 416 -> Convex Q6581097
	>>> Processing 419/9961: what killed tom held                                  istory of trunks fall into                                  
 419 -> Convex Q12192
	>>> Processing 421/9961: Where was pierre samuel du pont de nemours' place of birth?                                  
 421 -> qAnswer Q90

 421 -> Convex Q90
	>>> Processing 425/9961: where did archie maclaren pass away                                        
 425 -> Convex Q783210
	>>> Processing 429/9961: Where was yuriy furta birthed                                           

	>>> Processing 632/9961: Who is a child of pauline vanier                                                             
 632 -> Convex Q545644
	>>> Processing 643/9961: who is the author of empire of ivory?                                                            
 643 -> Convex Q242643
	>>> Processing 645/9961: what language was kamikaze 1989 filmed in                                   
 645 -> Convex Q188
	>>> Processing 647/9961: what musical genre is  syr8: andre sider af sonic youth                                    

In [14]:
#for r in df['mine']:
#    if r != (False, False):
#        print(r)

In [27]:
#df0_408_raw = df.copy()

In [ ]:
df0_408_raw

In [39]:
for r in df0_408['mine']:
    if r[1] == False:
        print(r)

(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, Fa

In [40]:
df0_408['mine'] = False

In [44]:
df

,question,source,qanswer,platypus,convex,mine
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,False
...,...,...,...,...,...,...
404,which band made on earth to make the numbers up,Q5473329,Q5473329,False,Q482994,False
405,Who's a defensive midfielder born in state of ...,Q17097400,False,False,Q12248982,False
406,what is the nationality of louise latimer?,Q30,Q30,False,Q145,False
407,where was harold gore's birthplace,Q49111,False,False,Q12018387,False


In [ ]:
#pickle_data(df, "wd-sq-test-qanswer-619-backup")